# Set initialization file
https://cccma.gitlab.io/classic/makeInputFiles.html

Initialize the model for CLASSIC (CLASS+CTEM) run. Use the Paul's SnowMIP data to set the first soil layers proporties
and then extrapolate with the satellite data. 

In [1]:
# Env: sc2_v0

import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import proplot as pplt # New plot library (https://proplot.readthedocs.io/en/latest/)
pplt.rc['savefig.dpi'] = 300 # 1200 is too big! #https://proplot.readthedocs.io/en/latest/basics.html#Creating-figures

In [2]:
exp = 'Ref'
site = 'wfj'
site_paul = 'Weissfluhjoch'
site_ex = 'CA-Oas'
# site_ex = 'AU-Tum'

path_in = '/home/lalandmi/eccc/classic-develop/inputFiles/FLUXNETsites/'+site_ex # example file
path_out = '/home/lalandmi/eccc/classic-develop/inputFiles/SnowMIP/'+site

In [3]:
ds = xr.open_dataset(path_in+'/'+site_ex+'_init.nc')
ds.load()

<xarray.Dataset>
Dimensions:            (tile: 1, lat: 1, lon: 1, icp1: 6, layer: 20, ic: 5,
                        icc: 12, iccp1: 13, iccp2: 14, slope: 8)
Coordinates:
  * ic                 (ic) int32 1 2 3 4 5
  * icc                (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * iccp1              (iccp1) int32 1 2 3 4 5 6 7 8 9 10 11 12 13
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * icp1               (icp1) int32 1 2 3 4 5 6
  * lat                (lat) float64 53.63
  * layer              (layer) int32 1 2 3 4 5 6 7 8 ... 13 14 15 16 17 18 19 20
  * lon                (lon) float64 -106.2
  * slope              (slope) int32 1 2 3 4 5 6 7 8
  * tile               (tile) int32 1
Data variables: (12/101)
    ALBS               (tile, lat, lon) float64 0.7647
    ALIC               (tile, icp1, lat, lon) float64 0.0 0.29 0.0 0.0 0.0 0.0
    ALVC               (tile, icp1, lat, lon) float64 0.0 0.05 0.0 0.0 0.0 0.0
    CLAY               (tile, layer, lat, lon) float64 15.16 20.08 ... 23.29
    CMAS               (tile, ic, lat, lon) float64 0.0 20.0 0.0 0.0 0.0
    Cmossmas           (tile, lat, lon) float64 0.0
    ...                 ...
    stemmass_s         (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    stmhrlos           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    tcoldm             (lat, lon) float32 99.0
    twarmm             (lat, lon) float32 -99.0
    tymaxlai           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nh4_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    title:          CLASSIC initialization file created from: /home/acrp001/m...
    creation_date:  20190523
    history:        Wed Jun 12 17:19:01 2019: ncks -4 ./rsfile.nc ./rsfile.nc
    NCO:            4.4.2
    row_bounds:     1 1

In [4]:
ds.soilcmas.sum()

<xarray.DataArray 'soilcmas' ()>
array(0.)

## Weissfluhjoch, Switzerland

Artificially drained moraine containing clay sediments overlain by serpentine rocks of
varying sizes up to 20 cm diameter.

| Short name | wfj |
|:-----------|:----|
| Location | 46.83ºN, 9.81ºE |
| Elevation | 2540 m |
| Snow-free albedo | 0.1 |
| Simulation period | 1 September 1996 to 31 August 2016 |
| Temperature/humidity measurement height | 4.5 m |
| Wind measurement height | 5.5 m |
| Reference | WSL ([2017](https://www.envidat.ch/dataset/snowmip)) |

In [5]:
# https://envidat.ch/#/metadata/snowmip
lat = 46.829598 
lon = 9.809568

# If the site description is not enough we can get the input data from gridded satellite datasets
path = '/home/lalandmi/Dropbox/data/CLASSIC/'
SoilGrids250m_CLAY = xr.open_dataset(path+'/soil/SoilGrids250m_CLAY_0.05deg.nc')
SoilGrids250m_ORGM = xr.open_dataset(path+'/soil/SoilGrids250m_ORGM_0.05deg.nc')
SoilGrids250m_SAND = xr.open_dataset(path+'/soil/SoilGrids250m_SAND_0.05deg.nc')
SoilGrids250m_SDEP = xr.open_dataset(path+'/soil/SoilGrids250m_SDEP_0.05deg.nc')
ESACCI_PFT_2000 = xr.open_dataset(path+'/ESACCI-LC-L4-PFT-Map-300m-P1Y-2000-v2.0.8.nc')
ESACCI_PFT_2010 = xr.open_dataset(path+'/ESACCI-LC-L4-PFT-Map-300m-P1Y-2010-v2.0.8.nc')
CTEM_12_PFTs_ESACCI_2010 = xr.open_dataset(path+'/CTEM_12_PFTs_ESACCI_2010_minus_water_rescaled_v2_1deg.nc')
NCAR_SOCI = xr.open_dataset(path+'/soil/NCAR_SOCI_0.5deg.nc')
c4_fraction_1deg = xr.open_dataarray(path+'/vegetation/c4_fraction_1deg.nc')

path_paul = '/home/lalandmi/Dropbox/data/SnowMIP/Paul' 
ds_init_Paul = xr.open_dataset(path_paul+'/'+site_paul+'_60_ESMSnowMIP.nc').load()
ds_init_Paul

<xarray.Dataset>
Dimensions:            (lon: 1, lat: 1, tile: 1, icp1: 5, layer: 3, ic: 4,
                        icctem: 9, iccp2: 11, slope: 8)
Coordinates:
  * lon                (lon) float64 9.81
  * lat                (lat) float64 46.83
  * tile               (tile) int32 1
  * icp1               (icp1) int32 1 2 3 4 5
  * layer              (layer) int32 1 2 3
  * ic                 (ic) int32 1 2 3 4
  * icctem             (icctem) int32 1 2 3 4 5 6 7 8 9
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11
  * slope              (slope) int32 1 2 3 4 5 6 7 8
Data variables: (12/43)
    GC                 (lat, lon) float64 -1.0
    nmtest             (lat, lon) float64 1.0
    FCAN               (tile, icp1, lat, lon) float64 0.0 0.0 0.0 0.0 0.0
    LNZ0               (tile, icp1, lat, lon) float64 0.0 0.0 0.0 0.0 0.0
    ALIC               (tile, icp1, lat, lon) float64 0.0 0.0 0.0 0.0 0.0
    ALVC               (tile, icp1, lat, lon) float64 0.0 0.0 0.0 0.0 0.0
    ...                 ...
    srplsmon           (lat, lon) float64 8.198e-319
    defctmon           (lat, lon) float64 7.996e-319
    anndefct           (lat, lon) float64 9.321e-314
    annsrpls           (lat, lon) float64 7.905e-323
    annpcp             (lat, lon) float64 0.0
    dry_season_length  (lat, lon) float64 0.0
Attributes:
    title:          CLASSIC initialization file created from: /home/lalandmi/...
    creation_date:  20240123

### Set the layer coords to the center soil layer

In [6]:
layer_c = []

for i in range(len(ds.DELZ)):
    if i == 0:
        layer_c.append(ds.DELZ.cumsum().values[i]/2)
    else:
        layer_c.append(ds.DELZ.cumsum().values[i-1] + ds.DELZ.values[i]/2)
        
with xr.set_options(keep_attrs=True):
    ds = ds.assign_coords(layer=ds.layer*0+layer_c)

ds.layer

<xarray.DataArray 'layer' (layer: 20)>
array([5.00e-02, 1.50e-01, 2.50e-01, 3.50e-01, 4.50e-01, 5.50e-01, 6.50e-01,
       7.50e-01, 8.50e-01, 9.50e-01, 1.10e+00, 1.35e+00, 1.70e+00, 2.15e+00,
       2.90e+00, 4.90e+00, 8.90e+00, 1.89e+01, 4.14e+01, 5.89e+01])
Coordinates:
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
Attributes:
    standard_name:  ground layers

In [7]:
ds.DELZ.cumsum()

<xarray.DataArray 'DELZ' (layer: 20)>
array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.2,
        1.5,  1.9,  2.4,  3.4,  6.4, 11.4, 26.4, 56.4, 61.4])
Coordinates:
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9

In [8]:
layer_c = []

for i in range(len(ds_init_Paul.DELZ)):
    if i == 0:
        layer_c.append(ds_init_Paul.DELZ.cumsum().values[i]/2)
    else:
        layer_c.append(ds_init_Paul.DELZ.cumsum().values[i-1] + ds_init_Paul.DELZ.values[i]/2)
        
with xr.set_options(keep_attrs=True):
    ds_init_Paul = ds_init_Paul.assign_coords(layer=ds_init_Paul.layer*0+layer_c)

ds_init_Paul.layer

<xarray.DataArray 'layer' (layer: 3)>
array([0.05 , 0.225, 2.225])
Coordinates:
  * layer    (layer) float64 0.05 0.225 2.225
Attributes:
    standard_name:  ground layers

In [9]:
ds_init_Paul.DELZ.cumsum()

<xarray.DataArray 'DELZ' (layer: 3)>
array([0.1 , 0.35, 4.1 ])
Coordinates:
  * layer    (layer) float64 0.05 0.225 2.225

### Combine Paul's data with satellite data (if needed)

Alpine tundra with thin soils and exposed bedrock.

#### SAND

In [11]:
ds_init_Paul.SAND.squeeze().values

array([90., 20., 20.])

In [12]:
SoilGrids250m_SAND.sel(lat=lat, lon=lon, method='nearest').SAND.values

array([36.596935, 42.42869 , 44.668198, 48.270798, 48.270798, 48.270798,
       51.03007 , 51.03007 , 51.03007 , 51.03007 , 52.35694 , 52.35694 ,
       52.35694 , 52.35694 , 52.35694 , 52.35694 , 52.35694 , 52.35694 ,
       52.35694 , 52.35694 ], dtype=float32)

In [13]:
SAND = ds_init_Paul.where(ds_init_Paul.SAND >= 0).interp(layer=ds.layer, method='linear').SAND.squeeze()
SAND = SAND.interpolate_na(dim='layer', method='nearest', fill_value="extrapolate")

# Deals only with the case of the first layer is peat
# (needs to be adapted if more peat layers)
if ds_init_Paul.SAND.squeeze().values[0] == -2:
    SAND.values[0] = -2 

# Average the last layers with satellite data
SAND.values[4:] = (SAND.values[4:] + SoilGrids250m_SAND.sel(lat=lat, lon=lon, method='nearest').SAND.values[4:])/2
SAND

<xarray.DataArray 'SAND' (layer: 20)>
array([90.        , 50.        , 20.        , 20.        , 34.13539886,
       34.13539886, 35.51503563, 35.51503563, 35.51503563, 35.51503563,
       36.17847061, 36.17847061, 36.17847061, 36.17847061, 36.17847061,
       36.17847061, 36.17847061, 36.17847061, 36.17847061, 36.17847061])
Coordinates:
    lon      float64 9.81
    lat      float64 46.83
    tile     int32 1
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
Attributes:
    units:      %
    long_name:  Percentage sand content

#### CLAY

In [14]:
ds_init_Paul.CLAY.squeeze().values

array([ 5., 60., 60.])

In [15]:
SoilGrids250m_CLAY.sel(lat=lat, lon=lon, method='nearest').CLAY.values

array([ 8.743296,  9.802126, 10.138506, 10.523492, 10.523492, 10.523492,
       10.734985, 10.734985, 10.734985, 10.734985, 10.832471, 10.832471,
       10.832471, 10.832471, 10.832471, 10.832471, 10.832471, 10.832471,
       10.832471, 10.832471], dtype=float32)

In [16]:
CLAY = ds_init_Paul.where(ds_init_Paul.SAND >= 0).interp(layer=ds.layer, method='linear').CLAY.squeeze()
CLAY = CLAY.interpolate_na(dim='layer', method='nearest', fill_value="extrapolate")

# Deals only with the case of the first layer is peat
# (needs to be adapted if more peat layers)
if ds_init_Paul.SAND.squeeze().values[0] == -2:
    CLAY.values[0] = 0

# Average the last layers with satellite data
CLAY.values[4:] = (CLAY.values[4:] + SoilGrids250m_CLAY.sel(lat=lat, lon=lon, method='nearest').CLAY.values[4:])/2
CLAY

<xarray.DataArray 'CLAY' (layer: 20)>
array([ 5.        , 36.42857143, 60.        , 60.        , 35.26174593,
       35.26174593, 35.36749268, 35.36749268, 35.36749268, 35.36749268,
       35.41623545, 35.41623545, 35.41623545, 35.41623545, 35.41623545,
       35.41623545, 35.41623545, 35.41623545, 35.41623545, 35.41623545])
Coordinates:
    lon      float64 9.81
    lat      float64 46.83
    tile     int32 1
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
Attributes:
    units:      %
    long_name:  Percentage clay content

#### ORGM

In [17]:
ds_init_Paul.ORGM.squeeze().values

array([0., 0., 0.])

In [18]:
SoilGrids250m_ORGM.sel(lat=lat, lon=lon, method='nearest').ORGM.values

array([25.057472 , 15.071839 , 11.839082 ,  8.49138  ,  8.49138  ,
        8.49138  ,  6.6522994,  6.6522994,  6.6522994,  6.6522994,
        5.804598 ,  5.804598 ,  5.804598 ,  5.804598 ,  5.804598 ,
        5.804598 ,  5.804598 ,  5.804598 ,  5.804598 ,  5.804598 ],
      dtype=float32)

In [19]:
ORGM = ds_init_Paul.where(ds_init_Paul.SAND >= 0).interp(layer=ds.layer, method='linear').ORGM.squeeze()
ORGM = ORGM.interpolate_na(dim='layer', method='nearest', fill_value="extrapolate")

# Deals only with the case of the first layer is peat
# (needs to be adapted if more peat layers)
if ds_init_Paul.SAND.squeeze().values[0] == -2:
    ORGM.values[0] = 0

# Average the last layers with satellite data
ORGM.values[4:] = (ORGM.values[4:] + SoilGrids250m_ORGM.sel(lat=lat, lon=lon, method='nearest').ORGM.values[4:])/2
ORGM

<xarray.DataArray 'ORGM' (layer: 20)>
array([0.        , 0.        , 0.        , 0.        , 4.24568987,
       4.24568987, 3.3261497 , 3.3261497 , 3.3261497 , 3.3261497 ,
       2.90229893, 2.90229893, 2.90229893, 2.90229893, 2.90229893,
       2.90229893, 2.90229893, 2.90229893, 2.90229893, 2.90229893])
Coordinates:
    lon      float64 9.81
    lat      float64 46.83
    tile     int32 1
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
Attributes:
    units:      %
    long_name:  Percentage organic matter content

In [20]:
ds_init_Paul.FCAN

<xarray.DataArray 'FCAN' (tile: 1, icp1: 5, lat: 1, lon: 1)>
array([[[[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]]])
Coordinates:
  * lon      (lon) float64 9.81
  * lat      (lat) float64 46.83
  * tile     (tile) int32 1
  * icp1     (icp1) int32 1 2 3 4 5
Attributes:
    units:      -
    long_name:  Annual maximum fractional coverage of modelled area (read in ...

In [21]:
ESACCI_PFT_2000.sel(lat=lat, lon=lon, method='nearest').load()

<xarray.Dataset>
Dimensions:       (time: 1, bounds: 2)
Coordinates:
    lon           float64 9.81
    lat           float64 46.83
  * time          (time) datetime64[ns] 2000-01-01
Dimensions without coordinates: bounds
Data variables: (12/20)
    WATER         (time) int8 0
    BARE          (time) int8 0
    BUILT         (time) int8 0
    GRASS-MAN     (time) int8 0
    GRASS-NAT     (time) int8 100
    SHRUBS-BD     (time) int8 0
    ...            ...
    TREES-NE      (time) int8 0
    LAND          (time) int8 100
    WATER_OCEAN   (time) int8 0
    lat_bounds    (bounds) float64 46.83 46.83
    lon_bounds    (bounds) float64 9.808 9.811
    time_bounds   (time, bounds) datetime64[ns] 2000-01-01 2000-12-31
Attributes: (12/40)
    title:                      ESA CCI Land Cover Project: Maps of Plant Fun...
    summary:                    This dataset contains the global plant functi...
    type:                       ESACCI-LC-L4-PFT-Map-300m-P1Y
    id:                         ESACCI-LC-L4-PFT-Map-300m-P1Y-2000-v2.0.8
    project:                    Medium Resolution Land Cover - Climate Change...
    references:                 https://maps.elie.ucl.ac.be/CCI/viewer/downlo...
    ...                         ...
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  002777777777778
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  002777777777778
    TileSize:                   2025:2025
    product_version:            2.0.8

In [23]:
ESACCI_PFT_2010.sel(lat=lat, lon=lon, method='nearest').load()

<xarray.Dataset>
Dimensions:       (time: 1, bounds: 2)
Coordinates:
    lon           float64 9.81
    lat           float64 46.83
  * time          (time) datetime64[ns] 2010-01-01
Dimensions without coordinates: bounds
Data variables: (12/18)
    WATER         (time) int8 0
    BARE          (time) int8 0
    BUILT         (time) int8 0
    GRASS-MAN     (time) int8 0
    GRASS-NAT     (time) int8 100
    SHRUBS-BD     (time) int8 0
    ...            ...
    TREES-BE      (time) int8 0
    TREES-ND      (time) int8 0
    TREES-NE      (time) int8 0
    lat_bounds    (bounds) float64 46.83 46.83
    lon_bounds    (bounds) float64 9.808 9.811
    time_bounds   (time, bounds) datetime64[ns] 2010-01-01 2010-12-31
Attributes: (12/37)
    title:                      ESA CCI Land Cover Project: Maps of Plant Fun...
    summary:                    This dataset contains the global plant functi...
    type:                       ESACCI-LC-L4-PFT-Map-300m-P1Y
    id:                         ESACCI-LC-L4-PFT-Map-300m-P1Y-2010-v2.0.8
    project:                    Climate Change Initiative - European Space Ag...
    references:                 http://www.esa-landcover-cci.org
    ...                         ...
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  002777777777778
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  002777777777778
    TileSize:                   2025:2025
    product_version:            2.0.8

In [24]:
CTEM_12_PFTs_ESACCI_2010.sel(lat=lat, lon=360+lon, method='nearest').load()

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    lon      float64 359.0
    lat      float64 46.5
Data variables: (12/13)
    NE       float32 0.002575
    ND       float32 0.0
    BE       float32 0.0
    BCD      float64 0.002276
    BDD      float64 0.0
    C3C      float32 0.876
    ...       ...
    C3G      float32 0.1072
    C4G      float32 0.0
    Sedge    float32 1.947e-09
    SBE      float32 3.894e-09
    SBD      float32 0.0005695
    BARE     float32 0.01141
Attributes:
    CDI:                       Climate Data Interface version 1.6.8 (http://m...
    Conventions:               CF-1.5
    cdo_openmp_thread_number:  8
    CDO:                       Climate Data Operators version 1.6.8rc2 (http:...
    created_by:                Ed Chan, CRD
    creation_date:             Wed Nov  9 19:52:31 GMT 2022
    provenance:                #1: https://climate.esa.int/en/projects/land-c...

In [25]:
# These values can be attributed with the global grid if not available
SDEP = SoilGrids250m_SDEP.sel(lat=lat, lon=lon, method='nearest').SDEP.values.item(0) # Soil permeable depth (m)
SOCI = NCAR_SOCI.sel(lat=lat, lon=lon, method='nearest').SOCI.values.item(0) # Soil color inde
c4_fraction = c4_fraction_1deg.sel(lat=lat, lon=lon, method='nearest').values.item(0) # If grass


# Those can be distributed along the soil layers if available
CLAY = CLAY.values
SAND = SAND.values
ORGM = ORGM.values
# If the sum does not reach 100 %, the left over will be attributed to silt

# ! List of CLASS-level PFTs
classpfts = ['NdlTr', 'BdlTr', 'Crops', 'Grass', 'BdlSh']
FCAN = {'NdlTr': 0. , 'BdlTr': 0., 'Crops': 0., 'Grass': 0., 'BdlSh': 0.} # max 1
# Bare ground

# ! List of CTEM PFTs
# ! **Note: 'BdlDCoTr' should be specified before 'BdlDDrTr' due to some code in competition.
ctempfts = ['NdlEvgTr', 'NdlDcdTr', 'BdlEvgTr', 'BdlDCoTr', 'BdlDDrTr', 'CropC3', 'CropC4', 'GrassC3', 'GrassC4', 'Sedge', 'BdlEvgSh', 'BdlDCoSh']
fcancmx = {'NdlEvgTr': 0., 'NdlDcdTr': 0., 'BdlEvgTr': 0., 'BdlDCoTr': 0., 'BdlDDrTr': 0., 'CropC3': 0., 'CropC4': 0., 
           'GrassC3': 0., 'GrassC4': 0., 'Sedge': 0., 'BdlEvgSh': 0., 'BdlDCoSh': 0.} # Max 1
# Check differences between C3 and C4 at Cdp? (TODO)

if sum(FCAN.values()) > 1: raise Exception("The sum of FCAN values needs to be lower than 1.")
if sum(fcancmx.values()) > 1: raise Exception("The sum of fcanmax values needs to be lower than 1.")

print('SDEP = ' + str(SDEP))
print('\nSOCI = ' + str(SOCI))
print('\nCLAY = ' + str(CLAY))
print('\nSAND = ' + str(SAND))
print('\nORGM = ' + str(ORGM))
print('\nGrass C4 fraction = ' + str(c4_fraction))

SDEP = 4.590000152587891

SOCI = 13.0

CLAY = [ 5.         36.42857143 60.         60.         35.26174593 35.26174593
 35.36749268 35.36749268 35.36749268 35.36749268 35.41623545 35.41623545
 35.41623545 35.41623545 35.41623545 35.41623545 35.41623545 35.41623545
 35.41623545 35.41623545]

SAND = [90.         50.         20.         20.         34.13539886 34.13539886
 35.51503563 35.51503563 35.51503563 35.51503563 36.17847061 36.17847061
 36.17847061 36.17847061 36.17847061 36.17847061 36.17847061 36.17847061
 36.17847061 36.17847061]

ORGM = [0.         0.         0.         0.         4.24568987 4.24568987
 3.3261497  3.3261497  3.3261497  3.3261497  2.90229893 2.90229893
 2.90229893 2.90229893 2.90229893 2.90229893 2.90229893 2.90229893
 2.90229893 2.90229893]

Grass C4 fraction = 0.014700000174343586


### Set lat/lon

In [26]:
with xr.set_options(keep_attrs=True):
    ds = ds.assign_coords(lat=(ds.lat*0+lat))
    ds = ds.assign_coords(lon=(ds.lon*0+lon))
ds

<xarray.Dataset>
Dimensions:            (tile: 1, lat: 1, lon: 1, icp1: 6, layer: 20, ic: 5,
                        icc: 12, iccp1: 13, iccp2: 14, slope: 8)
Coordinates:
  * ic                 (ic) int32 1 2 3 4 5
  * icc                (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * iccp1              (iccp1) int32 1 2 3 4 5 6 7 8 9 10 11 12 13
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * icp1               (icp1) int32 1 2 3 4 5 6
  * lat                (lat) float64 46.83
  * layer              (layer) float64 0.05 0.15 0.25 0.35 ... 18.9 41.4 58.9
  * lon                (lon) float64 9.81
  * slope              (slope) int32 1 2 3 4 5 6 7 8
  * tile               (tile) int32 1
Data variables: (12/101)
    ALBS               (tile, lat, lon) float64 0.7647
    ALIC               (tile, icp1, lat, lon) float64 0.0 0.29 0.0 0.0 0.0 0.0
    ALVC               (tile, icp1, lat, lon) float64 0.0 0.05 0.0 0.0 0.0 0.0
    CLAY               (tile, layer, lat, lon) float64 15.16 20.08 ... 23.29
    CMAS               (tile, ic, lat, lon) float64 0.0 20.0 0.0 0.0 0.0
    Cmossmas           (tile, lat, lon) float64 0.0
    ...                 ...
    stemmass_s         (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    stmhrlos           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    tcoldm             (lat, lon) float32 99.0
    twarmm             (lat, lon) float32 -99.0
    tymaxlai           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nh4_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    title:          CLASSIC initialization file created from: /home/acrp001/m...
    creation_date:  20190523
    history:        Wed Jun 12 17:19:01 2019: ncks -4 ./rsfile.nc ./rsfile.nc
    NCO:            4.4.2
    row_bounds:     1 1

### Set PFTs

In [27]:
for i, pft in enumerate(classpfts):
    ds.FCAN[0, i, 0, 0] = FCAN[pft]
    print(pft + ' -> ' + str(FCAN[pft]*100) + ' %')
    
ds.FCAN[0, :, 0, 0]
# 6th PFT: Bareground (need to be specified if CLASS used without CTEM)

NdlTr -> 0.0 %
BdlTr -> 0.0 %
Crops -> 0.0 %
Grass -> 0.0 %
BdlSh -> 0.0 %


<xarray.DataArray 'FCAN' (icp1: 6)>
array([0., 0., 0., 0., 0., 0.])
Coordinates:
  * icp1     (icp1) int32 1 2 3 4 5 6
    lat      float64 46.83
    lon      float64 9.81
    tile     int32 1
Attributes:
    units:      -
    long_name:  Annual maximum fractional coverage of modelled area (read in ...

In [28]:
for i, pft in enumerate(ctempfts):
    ds.fcancmx[0, i, 0, 0] = fcancmx[pft]
    print(pft + ' -> ' + str(fcancmx[pft]*100) + ' %')
    
ds.fcancmx[0, :, 0, 0]

NdlEvgTr -> 0.0 %
NdlDcdTr -> 0.0 %
BdlEvgTr -> 0.0 %
BdlDCoTr -> 0.0 %
BdlDDrTr -> 0.0 %
CropC3 -> 0.0 %
CropC4 -> 0.0 %
GrassC3 -> 0.0 %
GrassC4 -> 0.0 %
Sedge -> 0.0 %
BdlEvgSh -> 0.0 %
BdlDCoSh -> 0.0 %


<xarray.DataArray 'fcancmx' (icc: 12)>
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Coordinates:
  * icc      (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
    lat      float64 46.83
    lon      float64 9.81
    tile     int32 1
Attributes:
    units:      -
    long_name:  PFT fractional coverage per grid cell

### Set soil color

In [29]:
with xr.set_options(keep_attrs=True):
    ds['SOCI'][0, 0, 0] = SOCI
ds.SOCI

<xarray.DataArray 'SOCI' (tile: 1, lat: 1, lon: 1)>
array([[[13.]]])
Coordinates:
  * lat      (lat) float64 46.83
  * lon      (lon) float64 9.81
  * tile     (tile) int32 1
Attributes:
    units:      index
    long_name:  Soil colour index

### Set permeable depth

In [30]:
with xr.set_options(keep_attrs=True):
    ds['SDEP'][0, 0, 0] = SDEP
ds.SDEP

<xarray.DataArray 'SDEP' (tile: 1, lat: 1, lon: 1)>
array([[[4.59000015]]])
Coordinates:
  * lat      (lat) float64 46.83
  * lon      (lon) float64 9.81
  * tile     (tile) int32 1
Attributes:
    units:      m
    long_name:  Soil permeable depth

### Check maximum level before bedrock
https://gitlab.com/jormelton/classic/-/blob/develop/src/modelStateDrivers.f90?ref_type=heads#L968

In [31]:
ds.DELZ

<xarray.DataArray 'DELZ' (layer: 20)>
array([ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.2,
        0.3,  0.4,  0.5,  1. ,  3. ,  5. , 15. , 30. ,  5. ])
Coordinates:
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
Attributes:
    units:      m
    long_name:  Ground layer thickness

In [32]:
i = 0
while ds.DELZ.cumsum()[i] < SDEP:
    i += 1
    if i > ds.DELZ.shape[0]-1:
        print('The permable depth is greater than the model levels')
        break
        
maxlevel = i + 1 # first level of bedrock (next to the one containing SDEP)
maxlevel

16

In [33]:
ds.DELZ.cumsum()

<xarray.DataArray 'DELZ' (layer: 20)>
array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.2,
        1.5,  1.9,  2.4,  3.4,  6.4, 11.4, 26.4, 56.4, 61.4])
Coordinates:
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9

In [34]:
ds.DELZ.cumsum()[maxlevel]

<xarray.DataArray 'DELZ' ()>
array(11.4)
Coordinates:
    layer    float64 8.9

In [35]:
ds.DELZ.cumsum()[:maxlevel]

<xarray.DataArray 'DELZ' (layer: 16)>
array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.2, 1.5, 1.9,
       2.4, 3.4, 6.4])
Coordinates:
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 1.35 1.7 2.15 2.9 4.9

In [36]:
ds.DELZ.cumsum()[maxlevel:]

<xarray.DataArray 'DELZ' (layer: 4)>
array([11.4, 26.4, 56.4, 61.4])
Coordinates:
  * layer    (layer) float64 8.9 18.9 41.4 58.9

### Set soil contents and flags
Note: flags are only set in the SAND variable (-3 bedrock / -2 peatland)

In [37]:
with xr.set_options(keep_attrs=True):
    # Set values until bedrock
    ds.SAND[0, :maxlevel, 0, 0] = SAND[:maxlevel]
    ds.CLAY[0, :maxlevel, 0, 0] = CLAY[:maxlevel]
    ds.ORGM[0, :maxlevel, 0, 0] = ORGM[:maxlevel]
    
    # Set bedrock values
    ds.SAND[0, maxlevel:, 0, 0] = -3 # flag for bedrock
    ds.CLAY[0, maxlevel:, 0, 0] = 0.
    ds.ORGM[0, maxlevel:, 0, 0] = 0.

In [38]:
# Current values
ds.SAND[0, :, 0, 0]

<xarray.DataArray 'SAND' (layer: 20)>
array([90.        , 50.        , 20.        , 20.        , 34.13539886,
       34.13539886, 35.51503563, 35.51503563, 35.51503563, 35.51503563,
       36.17847061, 36.17847061, 36.17847061, 36.17847061, 36.17847061,
       36.17847061, -3.        , -3.        , -3.        , -3.        ])
Coordinates:
    lat      float64 46.83
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
    lon      float64 9.81
    tile     int32 1
Attributes:
    units:      %
    long_name:  Percentage sand content

In [39]:
ds.CLAY[0, :, 0, 0]

<xarray.DataArray 'CLAY' (layer: 20)>
array([ 5.        , 36.42857143, 60.        , 60.        , 35.26174593,
       35.26174593, 35.36749268, 35.36749268, 35.36749268, 35.36749268,
       35.41623545, 35.41623545, 35.41623545, 35.41623545, 35.41623545,
       35.41623545,  0.        ,  0.        ,  0.        ,  0.        ])
Coordinates:
    lat      float64 46.83
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
    lon      float64 9.81
    tile     int32 1
Attributes:
    units:      %
    long_name:  Percentage clay content

In [40]:
ds.ORGM[0, :, 0, 0]

<xarray.DataArray 'ORGM' (layer: 20)>
array([0.        , 0.        , 0.        , 0.        , 4.24568987,
       4.24568987, 3.3261497 , 3.3261497 , 3.3261497 , 3.3261497 ,
       2.90229893, 2.90229893, 2.90229893, 2.90229893, 2.90229893,
       2.90229893, 0.        , 0.        , 0.        , 0.        ])
Coordinates:
    lat      float64 46.83
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
    lon      float64 9.81
    tile     int32 1
Attributes:
    units:      %
    long_name:  Percentage organic matter content

In [41]:
ds

<xarray.Dataset>
Dimensions:            (tile: 1, lat: 1, lon: 1, icp1: 6, layer: 20, ic: 5,
                        icc: 12, iccp1: 13, iccp2: 14, slope: 8)
Coordinates:
  * ic                 (ic) int32 1 2 3 4 5
  * icc                (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * iccp1              (iccp1) int32 1 2 3 4 5 6 7 8 9 10 11 12 13
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * icp1               (icp1) int32 1 2 3 4 5 6
  * lat                (lat) float64 46.83
  * layer              (layer) float64 0.05 0.15 0.25 0.35 ... 18.9 41.4 58.9
  * lon                (lon) float64 9.81
  * slope              (slope) int32 1 2 3 4 5 6 7 8
  * tile               (tile) int32 1
Data variables: (12/101)
    ALBS               (tile, lat, lon) float64 0.7647
    ALIC               (tile, icp1, lat, lon) float64 0.0 0.29 0.0 0.0 0.0 0.0
    ALVC               (tile, icp1, lat, lon) float64 0.0 0.05 0.0 0.0 0.0 0.0
    CLAY               (tile, layer, lat, lon) float64 5.0 36.43 ... 0.0 0.0
    CMAS               (tile, ic, lat, lon) float64 0.0 20.0 0.0 0.0 0.0
    Cmossmas           (tile, lat, lon) float64 0.0
    ...                 ...
    stemmass_s         (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    stmhrlos           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    tcoldm             (lat, lon) float32 99.0
    twarmm             (lat, lon) float32 -99.0
    tymaxlai           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nh4_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    title:          CLASSIC initialization file created from: /home/acrp001/m...
    creation_date:  20190523
    history:        Wed Jun 12 17:19:01 2019: ncks -4 ./rsfile.nc ./rsfile.nc
    NCO:            4.4.2
    row_bounds:     1 1

## Save to netCDF
If CLASS would be run without CTEM further variables should be set. See: https://cccma.gitlab.io/classic/basicInputs.html -> Required vegetation data


In [42]:
ds.to_netcdf(path_out+'/'+site+'_init_spinup_'+exp+'.nc')
ds.to_netcdf(path_out+'/rsfile_spinup_'+exp+'.nc')

In [43]:
!mkdir -p /home/lalandmi/eccc/classic-develop/outputFiles/SnowMIP/{site}/spinup_{exp}